In [9]:
import cv2
import numpy as np

img_left = cv2.imread(r'D:\study\northGeom3d\data\calibrate\imgs\leftcamera\Cam2_12.jpg', cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread(r'D:\study\northGeom3d\data\calibrate\imgs\rightcamera\Cam1_12.jpg', cv2.IMREAD_GRAYSCALE)
# Создание объекта StereoBM (блочное сопоставление)
stereo = cv2.StereoBM_create(numDisparities=16*10, blockSize=21)

# Вычисление карты диспаритета
disparity = stereo.compute(img_left, img_right).astype(np.float32) / 16.0

# Параметры объектива MVL-LF8040M-F
focal_length = 80  # Фокусное расстояние объектива в мм (MVL-LF8040M-F)
baseline = 0.412  # Расстояние между камерами в метрах (пример, установите своё)
min_operation_distance = 0.230  # Минимальное рабочее расстояние объектива в метрах

# Преобразование фокусного расстояния в пиксели
# Обычно этот параметр зависит от параметров камеры (матрицы). Пример для матрицы с размером пикселя:
sensor_pixel_size = 0.005  # Размер пикселя матрицы в мм (пример)
focal_length_px = focal_length / sensor_pixel_size  # Преобразование в пиксели

# Вычисление карты глубины
depth_map = (focal_length_px * baseline) / (disparity + 1e-6)  # Добавляем малое значение, чтобы избежать деления на 0

# Ограничиваем значения карты глубины, чтобы исключить слишком близкие или далёкие объекты
depth_map[depth_map > 100] = 100  # Устанавливаем порог глубины в 100 метров (пример)
depth_map[depth_map < min_operation_distance] = min_operation_distance  # Учитываем минимальное рабочее расстояние объектива

# Нормализация карты глубины для визуализации
depth_map_vis = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
depth_map_vis = np.uint8(depth_map_vis)

# Визуализация карты глубины
depth_map_vis = cv2.resize(depth_map_vis,(512,512))
cv2.imshow('Depth Map', depth_map_vis)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
import cv2
import numpy as np

# Функция обновления карты глубины
def update_depth_map(val):
    # Получение текущих значений с трекбаров
    numDisparities = cv2.getTrackbarPos('Num Disparities', 'Depth Map') * 16  # numDisparities должно быть кратно 16
    blockSize = cv2.getTrackbarPos('Block Size', 'Depth Map') * 2 + 5  # Должно быть нечетным и >= 5

    # Обновляем объект StereoBM с новыми параметрами
    stereo = cv2.StereoBM_create(numDisparities=numDisparities, blockSize=blockSize)

    # Вычисление карты диспаритета
    disparity = stereo.compute(img_left, img_right).astype(np.float32) / 16.0

    # Вычисление карты глубины
    depth_map = (focal_length_px * baseline) / (disparity + 1e-6)

    # Ограничиваем значения карты глубины
    depth_map[depth_map > 100] = 100
    depth_map[depth_map < min_operation_distance] = min_operation_distance

    # Нормализация карты глубины для визуализации
    depth_map_vis = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map_vis = np.uint8(depth_map_vis)

    # Масштабируем и отображаем карту глубины
    depth_map_vis = cv2.resize(depth_map_vis, (512, 512))
    cv2.imshow('Depth Map', depth_map_vis)

# Чтение изображений с двух камер
img_left = cv2.imread(r'D:\study\northGeom3d\data\calibrate\imgs\leftcamera\Cam2_12.jpg', cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread(r'D:\study\northGeom3d\data\calibrate\imgs\rightcamera\Cam1_12.jpg', cv2.IMREAD_GRAYSCALE)

# Параметры объектива MVL-LF8040M-F
focal_length = 80  # Фокусное расстояние объектива в мм
baseline = 0.412  # Расстояние между камерами в метрах
min_operation_distance = 0.230  # Минимальное рабочее расстояние объектива в метрах

# Преобразование фокусного расстояния в пиксели
sensor_pixel_size = 0.005  # Размер пикселя матрицы в мм
focal_length_px = focal_length / sensor_pixel_size  # Преобразование фокусного расстояния в пиксели

# Создание окна для отображения карты глубины
cv2.namedWindow('Depth Map')

# Создание трекбаров для настройки параметров
cv2.createTrackbar('Num Disparities', 'Depth Map', 10, 16, update_depth_map)  # numDisparities от 10*16 до 16*16
cv2.createTrackbar('Block Size', 'Depth Map', 8, 25, update_depth_map)  # blockSize от 5 до 51

# Инициализация карты глубины с начальными параметрами
update_depth_map(0)

# Ожидание клавиши для выхода
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:862: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'
